In [4]:
import numpy as np
import pandas as pd               
from scipy.special import factorial
import fileinput

## Information:
1. The file is about calculating the ''layer coefficients'' from a given measured map, such as *'ETM05_S1_finesse'*

2. The layer coefficients calculated here are used to generate random maps that has the same layer coefficients (i.e. same spatial frequency content) (in notebook *''Random maps generation with ETM05_S1_finesse.ipynb''* ) 

3. To test that the functions in this notebook, a test file 'testing.txt' is generated in *Random maps generation with ETM05_S1_finesse.ipynb* with the fake layer coefficients [0,0,1,2,3,4,5,6,7,8]. Decomposing the testing map in this noteook, the resulting coefficients are close to [0,0,1,2,3,4,5,6,7,8], see below.

In [5]:
# return a list of Zernike maps(basis) for aLIGO maps to project to
def Zernikes(shape, radius, step_size, max_zern):
    center = (shape-1)/2 # Center of the map
    rrange = radius/step_size  # Range of the map such that the radius is 0.15 for a given step_size
    zernikes=[] # list of Zernike maps

    # This is the same set of functions in the Zernike maps generation notebook
    def theta(x,y):
        phi = np.arctan2(y, x)
        return phi

    def radial(x,y,n,m):
        if m<0:
            m=-m
        sum=0
        for k in range(int((n-m)/2)+1):   
            r=(-1)**k*factorial(n-k)/factorial(k)/factorial((n+m)/2-k)/factorial((n-m)/2-k)*((x**2+y**2)/(rrange**2))**(n/2-k)
            # Here I used '(x**2+y**2)/(rrange**2)' instead of just '(x**2+y**2)' to normalize the radius
            sum+=r
        return sum

    def angular(x,y,n,m): 
        a=theta(x,y)
        if m>=0:
            angular=np.cos(m*a)
        else:
            angular=-np.sin(m*a)
        return angular
    
    for n in range(max_zern):
        for m in range(-n,n+1,2):
            stepRange = np.arange(shape)-center
            x,y=np.meshgrid(stepRange,stepRange,sparse=True)
            zfunc=radial(x,y,n,m)*angular(x,y,n,m)
            for i in range(shape):
                for j in range(shape): 
                    if (i-center)**2+(j-center)**2>= rrange**2:
                        zfunc[i][j]=0  # Set the values outside the cropping radius to zero
            zmap=zfunc/np.abs(zfunc).max() # Such that the amplitude(maximum value in the map data) equals to 1
            zernikes.append(zmap)
    return zernikes # Return a list of Zernike maps

In [6]:
def layerCoeffs(filename, zernikebasis, order=10):
    i=0 # Create a conversion between {n}{m} to {i}
    dic={}
    for n in range(order):
        for m in range(-n,n+1,2):
            dic[f'{n}{m}']=i
            i=i+1

    aLIGO=pd.read_csv(filename, header=None, sep=" ", skiprows=9).dropna(axis=1).values
    
    layerCoeff=[0,0] # Two zeros represent the first two layer coefficients
    for n in range(2,order): # Start from the third layer
        layer=0
        for m in range(-n,n+1,2):
            index=dic[f'{n}{m}']
            coeff=((aLIGO*zernikebasis[index]).sum())/((zernikebasis[index]**2).sum())
            if n==2 and m==0: # The coefficient of Z20 will not be added to the layer coefficient since it can be removed manually
                continue
            else:
                layer+=coeff**2
        layerCoeff.append(np.sqrt(layer))
    return layerCoeff

In [7]:
# generate the zernike basis, this takes a long time to run
basis_10 = Zernikes(1131,0.15,0.0002669951063580811,10)
basis_25 = Zernikes(1131,0.15,0.0002669951063580811,25)

In [8]:
# Testing of random map generation
# the 'testing.txt' was generated in the random maps generation file with layer coefficients of [0,0,1,2,3,4,5,6,7,8]
layerCoeffs('testing.txt', basis_10)
# and we do get [0,0,1,2,3,4,5,6,7,8]

[0,
 0,
 0.9991775923262345,
 1.9993048676357508,
 2.999679396670623,
 4.000356833087034,
 4.999081659420682,
 5.994341533038726,
 6.999764940271446,
 7.999018210477589]

In [11]:
layerCoeffs('ETM05_S1_finesse.txt', basis_25, order=25)

# Use these set of values to generate the random maps

[0,
 0,
 0.20190713375335295,
 0.14589743213720582,
 0.13728131907327343,
 0.06476026222531273,
 0.06396438434566959,
 0.061635581927653296,
 0.11786865519746435,
 0.10733282124903534,
 0.07524138634759972,
 0.046375544797053296,
 0.05357200662514628,
 0.0878930592250278,
 0.07583595265248438,
 0.0749722539467617,
 0.07595435402124659,
 0.06254947966202423,
 0.08196084165372425,
 0.08585295172293457,
 0.08870503602805532,
 0.09579683509187605,
 0.08575422531961867,
 0.07738351927467899,
 0.07309294568711436]